# sklearn and stochastic gradient descent

Until now we've been using the more standard LassoCV, RidgeCV, etc. (or GridSearchCV) to find our optimal parameters. Unfortunately, though these methods work well on smaller datasets with relatively small numbers of columns, once you start getting into "Medium Data" these slow down to a crawl, and take up so much memory that fitting them becomes untenable.

This is where stochastic gradient descent comes in. Because of its ability to fit iteratively on portions of the data, it avoids the issue of large datasets. It is the most common algorithm to fit models on large datasets.

---

### Import the packages

In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd

import patsy

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

---

### Load the data

I've provided data from the SF assessor's office on housing prices in San Francisco - already cleaned up. However, if you want to try this out on data you've been having trouble fitting, such as the Yelp data for project 5, feel free!


In [2]:
prop = pd.read_csv('/Users/Vrushank/Desktop/DSI-SF-3/datasets/sf_assessor_value/assessor_value_cleaned.csv')

---

### Sample down the data

For demonstration purposes and the sake of speed, I am sampling down this large dataset to a more reasonable amount of rows. 

Stochastic gradient descent is much faster, but the real benefit in my opinion is that it can fit much larger datasets. That being said, it is still slow to fit on huge datasets in sklearn. I don't recommend fitting on the entire data. This is actually my recommendation in general; finding the optimal parameters with more and more data will change the hyperparameters but often with marginal returns.

In [3]:
prop_sample = prop.sample(frac=0.1)
prop_sample.shape

(75415, 17)

---

### SGD with regression

Below I set up X, y data predicting value (housing price) from the remaining variables. There are ~75,000 rows, with 170 columns.

In [4]:
prop_sample_dummy = pd.get_dummies(prop_sample, columns=['zone', 'neighborhood', 'property_class'], dummy_na=False)

In [5]:
X_prop = prop_sample_dummy.drop('value', axis=1).values
y_prop = prop_sample_dummy.value

---

### Import the modeling classes

`SGDRegressor` and `SGDClassifer` are the models used in this solution These are the very general, flexible stochastic gradient descent classes.

In [6]:
from sklearn.linear_model import (LinearRegression, LogisticRegression, 
                                  Lasso, Ridge,
                                  SGDRegressor, SGDClassifier)
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

import scipy.stats as stats


---

### Standardize the data

Always a necessary step when performing regularization.

In [8]:
ss = StandardScaler()
Xn = ss.fit_transform(X_prop)

print Xn.shape

(75415, 173)


---

### Run the gridsearch on parameters

SGDRegressor and SGDClassifier use GridSearchCV and RandomizedSearchCV (which will be introduced in the next section) to find the optimal parameters. If you are still unsure of how to use the GridSearchCV, please look this up in the sklearn documentation. I want you to get used to looking things up online - you will be doing this literally every day on the job.

In [24]:
sgd_params = {
    'loss': ['squared_loss'],
    'penalty': ['l1', 'l2'],
    'alpha': np.logspace(-4, 10)
}

sgd_reg = SGDRegressor()
sgd_reg_gs = GridSearchCV(sgd_reg, sgd_params, cv=3, verbose=2, n_jobs=-1)

In [25]:
sgd_reg_gs.fit(Xn, y_prop)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/Vrushank/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] penalty=l1, alpha=0.0001, loss=squared_loss .....................


/Users/Vrushank/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] penalty=l1, alpha=0.0001, loss=squared_loss .....................


...

[CV] ..... penalty=l1, alpha=10000000000.0, loss=squared_loss -   1.3s
[CV] penalty=l2, alpha=10000000000.0, loss=squared_loss ..............


/Users/Vrushank/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] ..... penalty=l1, alpha=10000000000.0, loss=squared_loss -   1.4s
[CV] penalty=l2, alpha=10000000000.0, loss=squared_loss ..............
[CV] ..... penalty=l2, alpha=10000000000.0, loss=squared_loss -   0.9s
[CV] ..... penalty=l1, alpha=10000000000.0, loss=squared_loss -   1.3s
[CV] ..... penalty=l2, alpha=10000000000.0, loss=squared_loss -   0.8s
[CV] ..... penalty=l2, alpha=10000000000.0, loss=squared_loss -   0.7s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['squared_loss'], 'alpha': array([  1.00000e-04,   1.93070e-04,   3.72759e-04,   7.19686e-04,
         1.38950e-03,   2.68270e-03,   5.17947e-03,   1.00000e-02,
         1.93070e-02,   3.72759e-02,   7.19686e-02,   1.38950e-01,
         2.68270e-01,   5.1...    3.72759e+08,   7.19686e+08,   1.38950e+09,   2.68270e+09,
         5.17947e+09,   1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=2)

---

### Visualize or otherwise look at the SGDRegression results

How you choose to examine the results is up to you. Visualizations are always a good idea, but not entirely neccessary (or easy) in this case.

In [26]:
print sgd_reg_gs.best_params_
print sgd_reg_gs.best_score_

{'penalty': 'l2', 'alpha': 1.0, 'loss': 'squared_loss'}
0.216721388002


In [27]:
sgd_reg_gs_params = sgd_reg_gs.get_params()
sgd_reg_gs_params

{'cv': 3,
 'error_score': 'raise',
 'estimator': SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
        fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
        loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
        random_state=None, shuffle=True, verbose=0, warm_start=False),
 'estimator__alpha': 0.0001,
 'estimator__average': False,
 'estimator__epsilon': 0.1,
 'estimator__eta0': 0.01,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.15,
 'estimator__learning_rate': 'invscaling',
 'estimator__loss': 'squared_loss',
 'estimator__n_iter': 5,
 'estimator__penalty': 'l2',
 'estimator__power_t': 0.25,
 'estimator__random_state': None,
 'estimator__shuffle': True,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'fit_params': {},
 'iid': True,
 'n_jobs': -1,
 'param_grid': {'alpha': array([  1.00000000e-04,   1.93069773e-04,   3.72759372e-04,
           7.19685673e-04,   1.38949549e-03,   2.68269580e-03,
           5.1794

In [30]:
def lasso_coefs(X, y, alpha):
    coefs = []
    lasso_reg = Lasso()
    for a in alpha:
        lasso_reg.set_params(alpha=a)
        lasso_reg.fit(X, y)
        coefs.append(lasso_reg.coef_)
    return coefs 

sgd_reg_alphas = sgd_reg_gs_params['param_grid']['alpha']
lcoefs = lasso_coefs(Xn, y_prop, sgd_reg_alphas)

/Users/Vrushank/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


---

### RandomizedSearchCV

This class is very similar to GridSearchCV in the way it is initialized with parameters. The big difference is that instead of searching across a strictly specified grid, it searches over random values that are defined by distributions.

Below I have set up for you an example of parameters and calls to the class.

    uniform: this random variable is from scipy.stats and it pulls random values from the uniform distribution from 0.01 to 20000
    sgd_rand_params: the only difference here is that alpha gets the random variable. It will pull random values from that distribution
    RandomizedSearchCV: this takes an n_iter parameter that specifies how long to search over random values
    
RandomizedSearchCV is often faster than GridSearchCV while getting the same optimal parameteres. However, this is not _always_ the case. As far as I know it is not completely random, but rather begins to favor values that are closer to its current optimum.

For more information, see:

[RandomizedSearchCV documentation](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html#sklearn.grid_search.RandomizedSearchCV)

[scipy.stats available distributions and functions](http://docs.scipy.org/doc/scipy/reference/stats.html)


In [4]:
uniform = stats.uniform(loc=0.01, scale=20000)

sgd_rand_params = {
    'loss':['squared_loss'],
    'penalty':['l1','l2'],
    'alpha':uniform
}

sgd_reg = SGDRegressor()
sgd_reg_rand_gs = RandomizedSearchCV(sgd_reg, sgd_rand_params, cv=5, verbose=2, n_iter=50)

---

### Visualize/examine the results from your RandomizedSearchCV results

---

### SGDClassifier

Using either GridSearchCV or RandomizedSearchCV, set up an X, y classification problem to look at. Depending on the data you loaded in, you may need to get new data. 

Find the optimal parameters.

---

### Visualize/examine the results of the classifier

Just like above.